# The `Utils` notebook  <a id="Utils"></a>  [<font size=1>(back to `Main.ipynb`)</font>](./Main.ipynb)

This notebook implements a number of useful (though not essential) functions.

## The notebook is organized as follows:

* a [norm](#norm) function;
* a [linear interpolation](#linear-interp) function;
* a [function zeroing](#trunc-mat) all values below a threshold;
* a [Gini](#Gini) function;
* functions for [pretty printing](#pretty-print);
* some functions related to [agents remaining in the same state](#same-state-funs) for a number of periods

It also contains a number of [deprecated functions](#deprecated), kept for legacy reasons.

# The norm function  <a id="norm"></a>[<font size=1>(back to menu)</font>](#Utils)

We construct a simple norm function, whose signature is:

> `norm(x::AbstractArray; p::Union{Nothing,Int}=nothing)`.

It returns the supremum norm if `p=nothing` and the p-norm otherwise.

In [5]:
function norm(x::AbstractArray; p::Union{Nothing,Int}=nothing) 
    return isnothing(p) ? maximum(abs.(x)) : (sum(abs.(x).^p))^(1/p)
end;

In [3]:
norm([-1,2,-10])

10

In [4]:
norm([-1,2,-10])

10

# The linear interpolation function  <a id="linear-interp"></a>[<font size=1>(back to menu)</font>](#Utils)


The function `interpLinear` is a linear interpolation function, that is more specialized and hence slightly faster than the interpolation function of the `LinearInterpolation` package.

The function specification is: 
> `interpLinear(x::T,xs::Vector{T},ys::Vector{T},n::I,lb_y::T)`, 

where:
* `x` is a scalar where the interpolation is computed;
* `xs` and `ys` are sorted vectors such that `(xs[i], ys[i])` for `i=1,...,n` is a set of `n` known points;
* `n` is the common length of vectors `xs` and `ys`;
* `lb` is the lower bound for the truncation of the interpolation. If the interpolation result is lower than lb, the interpolation is trauncated to lb.

The function returns a pair containing:
* a scalar corresponding to the interpolated value (truncated if needed);
* an index `np` which is such that the interpolation is performed between indices `np` and `np+1`.

In [ ]:
function interpLinear(x::T, 
                      xs::Vector{T},
                      ys::Vector{T}, 
                      n::I, lb_y::T)::Tuple{T,I} where{T<:Real,I<:Integer}
   
    nx = searchsortedlast(xs, x) 
    #nx is the largest index such that xs[nx]≤x (and hence xs[nx+1]>x). Returns 0 if x≤xs[1]. xs sorted.
        
    #Adjust nx if x falls out of bounds of xs
    if nx == 0
        nx += 1
    elseif (nx==n)
        nx -= 1
    end    

    # Linear interpolation in x between xs[nx] and xs[nx+1]
    x_l,x_h = xs[nx],xs[nx+1]
    y_l,y_h = ys[nx],ys[nx+1]
    y = y_l + (y_h-y_l)/(x_h-x_l)*(x-x_l) 
    
    # returns interpolated value and corresponding index
    return ((y<lb_y) ? lb_y : y), nx
    
end

# Truncating stochastic matrix  <a id="trunc-mat"></a>[<font size=1>(back to menu)</font>](#Utils)


The function `truncMatrix` is function, that sets to zero all values of a matrix below a given threshold. 
The lost mass is attributed to the diagonal. The function only works for *square* matrices, and raises an error otherwise. 

The function specification is: 
> `truncMatrix(M::AbstractMatrix{T},threshold::T=1e-5)::AbstractMatrix{T} where {T<:Real}`, 

where:
* `M` is an abstract matrix;
* `threshold` is the threshold below which values are set to 0.

The function returns a matrix of the same size as the input.

In [68]:
function dimStoch(M::AbstractMatrix{T})::Integer  where {T<:Real}
    n,p = size(M)
    if all(sum(M,dims=2) .≈ ones(eltype(M),size(sum(M,dims=2))))
        return 2
    elseif all(sum(M,dims=1) .≈ ones(eltype(M),size(sum(M,dims=1))))
        return 1
    else
        return 0
    end
end

function truncMatrix(M::AbstractMatrix{T},threshold::T)::AbstractMatrix{T} where {T<:Real}
    M_ = copy(M)
    M_[M_.<=threshold].=zero(T)
    (n,p) = size(M)
    dim_stoch = dimStoch(M)
    @assert (n==p)&&(dim_stoch>0)
    for i = 1:n
        M_[i,i] += one(T) - sum(M_,dims=dim_stoch)[i]
    end
    return M_
end

function truncMatrix(M::AbstractMatrix{T};threshold::T=1e-5)::AbstractMatrix{T} where {T<:Real}
    return truncMatrix(M,threshold)
end;

truncMatrix (generic function with 2 methods)

In [72]:
M = rand(5,5)
#truncMatrix(M)


LoadError: AssertionError: n == p && dim_stoch > 0

# Bound array <a id="my-bound"></a>[<font size=1>(back to menu)</font>](#Utils)

The function `myBound` returns an array where all elements are bounded.

The function specification is: 
> `myBound(A::AbstractArray{T},inf::T,sup::T)::AbstractArray{T} where {T<:Real}`, 

where:
* `A` is an abstract array;
* `inf` is the lower bound;
* `sup` is the upper bound;

The function returns an array similar to the input, but where all elements are bounded by `inf` and `sup`.

In [ ]:
function myBound(A::AbstractArray{T},inf::T,sup::T)::AbstractArray{T} where {T<:Real}
    toR = similar(A)
    for i in eachindex(A)
        toR[i] = min(sup,max(inf,A[i]))
    end
    return toR
end;

In [ ]:
A = [-1, 1, 2, 3]
myBound(A, 0, 1)
A = [-.1 .1; 2. .3]
myBound(A, 0., 1.)

# Gini coefficient  <a id="Gini"></a>[<font size=1>(back to menu)</font>](#Utils)

We consider a discrete distribution of variables $\{y_i\}_{i=1,\ldots,n}$ with probabilities $\{p_{y_i}\}_{i=1,\ldots,n}$. The variables $\{y_i\}$ are positive and sorted in increasing order ($0<y_i<y_{i+1}$). 
The Gini coefficient $G$ is then defined as:
$$G = 1- \frac{\sum_{i=1}^n p_{y_i}(S_{i-1}+S_{i})}{S_n},$$
where $S_0=0$ and:
$$S_i = \sum_{j=1}^i y_jp_{y_j}.$$

The function signature is:
> `Gini(ys::Vector{T}, pys::Vector{T})::T`

or 

> `Gini(ys::Matrix{T}, pys::Matrix{T})::T`

where: 
* `ys` corresponds to $\{y_i\}_{i=1,\ldots,n}$ and is not required to be sorted, 
* `pys` to $\{p_{y_i}\}_{i=1,\ldots,n}$. 

The inputs `ys` and `pys` can be either of type `Vector` or `Matrix` (but both of the same type).

In [27]:
function Gini(ys::Vector{T}, pys::Vector{T})::T where{T<:Real}
    @assert size(ys)==size(pys)
    iys = sortperm(ys)
    ys .= ys[iys]
    pys .= pys[iys]
    Ss = [zero(T); cumsum(ys.*pys)]
    return one(T) - sum(pys.*(Ss[1:end-1].+Ss[2:end]))/Ss[end]
end
function Gini(ys::Matrix{T}, pys::Matrix{T})::T where{T<:Real}
    @assert size(ys)==size(pys)
    return Gini(ys[:],pys[:])
end

Gini (generic function with 2 methods)

# Pretty printing <a id="pretty-print"></a>[<font size=1>(back to menu)</font>](#Utils)

For [dictionaries](#print-dict) and [arrays](#print-array).

## Printing dictionaries <a id="print-dict"></a>[<font size=1>(back to pretty printing)</font>](#pretty-print)

The function 
> `print_dict(dict::Dict{String,T}; sep=':',digits::Int64=4)::Nothing`

prints the description (label$\,\times\,$value) in the alphabetical order of the labels, with `digits` digits for the rounding and where label and value are separeted by `sep`. 

In [ ]:
function print_dict(dict::Dict{String,T}; sep="",digits=4)::Nothing where {T<:Real}
    tuples = sort([(k,v) for (k,v) in dict], by = first)
    max_label_length = maximum(map(length∘first,tuples))
    for (k,v) in tuples
        k_ = lowercase(k)
        trail_space = repeat(' ',1+max_label_length-length(k))
        to_pct = (occursin("to-gdp",k_)||occursin("share",k_)) 
        if to_pct
            println(k,sep*trail_space,round(100*v,digits=digits-2),"%")
        else
            println(k,sep*trail_space,round(v,digits=digits))
        end
    end
    return nothing
end;

## Printing arrays <a id="print-array"></a>[<font size=1>(back to pretty printing)</font>](#pretty-print)

The function 
> `print_array(M::AbstractMatrix{T};digits::Int64=4)::Nothing where {T<:Real}`

prints the matrix `M` in a proper way (rows and then columns). Decimals are aligned on the decimal separator.

In [71]:
function print_array(M::AbstractMatrix{T};sep=" ",digits::Int64=4)::Nothing where {T<:Real}
    Ms = map(x -> string(round(x,digits=digits)), M)
    function length_dec(cs)
        css = split(cs,".")
        ncss = length(css)
        if ncss == 1
            return 0
        elseif ncss == 2
            return length(css[2])
        else
            @error("problem in splitting the decimal part of $cs")
        end
    end
    function length_int(cs)
        return length(split(cs,".")[1])
    end
    n_dec_max = maximum(length_dec.(Ms))
    n_int_max = maximum(length_int.(Ms))
    for i in eachindex(M[:,1])
        s = ""
        for j in eachindex(M[1,:])
            s *= repeat(" ",n_int_max-length_int(Ms[i,j]))*Ms[i,j]*repeat(" ",n_dec_max-length_dec(Ms[i,j]))*sep
        end
        println(s[1:end-length(sep)])
    end
    return nothing
end;

# Same-state functions  <a id="same-state-funs"></a>[<font size=1>(back to menu)</font>](#Utils)

## Accounting for the severance pay and the severance periods <a id="eco-severance"></a> [<font size=1>(back to `Economy`)</font>](#input-structures)

The function 

> `raw_to_full(y_path_raw,economy::Economy)`

take a path of *raw* income  indices (typically employment and unemployment) and returns a path of income indices with severance periods. 

For example, $[2,2,1]$ (employed for two periods and then unemployed for one period) will become $[5,5,4,3,2,1]$ with the former calibration (3 severance periods). See [example](#example-severance) below.

In [2]:
function raw_to_full(y_path_raw::Vector{I},economy::Economy{T,I,T1}) where {T<:Real,I<:Int,T1<:Function}
    @unpack y_size_raw,times_to_u, y_size = economy
    
    if any(y_path_raw .> y_size_raw)
        @info("The initial raw path includes raw states above the limit. Indices: ", 
            collect(1:length(y_path_raw))[y_path_raw .> y_size_raw])
        y_path_raw[y_path_raw .> y_size_raw] .= y_size_raw
    end
    
    n_transiton_to_u = sum([y_path_raw[i-1]>1&&y_path_raw[i]==1 for i = 2:length(y_path_raw)])
    y_path_full = zeros(eltype(y_path_raw),length(y_path_raw) + times_to_u*n_transiton_to_u)
    
    function raw_to_full_index(y_index;t=0,transition=false)
        if y_index == 1 && !transition 
            return 1
        elseif (t>times_to_u) 
            return 1 
        elseif !(transition)
            return min(1+y_index-1+times_to_u,y_size)
        else
            return min(1+(1-t)+(y_index-1)*times_to_u,y_size)
        end
    end
    y_path_full[1] = raw_to_full_index(y_path_raw[1])
    i_full = 2
    for i_raw in 2:length(y_path_raw)
        
        if y_path_raw[i_raw-1]>1 && y_path_raw[i_raw]==1
            for t = 1:times_to_u
                y_path_full[i_full+t-1] = raw_to_full_index(y_path_raw[i_raw-1],t=t,transition=true)
            end
            y_path_full[i_full+times_to_u] = raw_to_full_index(y_path_raw[i_raw])
            i_full += times_to_u+1
        else
            y_path_full[i_full] = raw_to_full_index(y_path_raw[i_raw])
            i_full += 1
        end
    end
    return y_path_full
end;

LoadError: LoadError: UndefVarError: @unpack not defined
in expression starting at In[2]:2

#### Example <a id="example-severance"></a>

Illustration of the example described [here](#eco-severance).

In [ ]:
#raw_to_full([2,2,1],economy)

## Finding the index for a value between the two indices

The function 
> `myfindbtwn(x,xs)`
returns the first index `i` such that `xs[i] ≤ x < xs[i+1]`. It returns `1` if `x < xs[1]` and `length(xs)-1`  if `x ≥ xs[end]`

#### *Remarks.*
* The array `xs` is assumed to be sorted in increasing order. 
* The function *always* returns an index `i` whic is a choice dictated by its usage in `transitionMat`.

In [ ]:
# returns the index i such that: xs[i]≤x<xs[i+1]
# xs is assumed to be sorted in increasing order
# we impose bouds on i such that i≥1 and i+1≤length(xs)
function myfindbtwn(x,xs)
    i = findfirst(xs.≥x)
    if isnothing(i)
        return length(xs)-1
    elseif i==1
        return i
    else
        return i-one(typeof(i))
    end
end;

#### *Example.*

In [ ]:
# expected result: (1, 2, 3)
myfindbtwn(0,[1,2,4,10]), myfindbtwn(3,[1,2,4,10]), myfindbtwn(10,[1,2,4,10])

## The `paths` function <a id="application-paths"></a> [<font size=1>(back to applications)</font>](#application)

The function 
> `paths(y_path::Vector{I},a0::T,c0::T,solution::AiyagariSolution,economy::Economy{T,I,T1})`

computes the paths of individual choices corresponding to the income path `y_path`, the policy and value functions of the solution `solution`, and the economy `economy`.

The income path `y_path` must include severance elements (via the function [`raw_to_full`](#eco-severance) for instance).

The function returns a tuple `a_path,c_path` consisting of the paths of savings and consumption decisions. 

In [ ]:
function paths(y_path::Vector{I},a0::T,c0::T,solution::AiyagariSolution,economy::Economy{T,I,T1,T2,T3}
        ) where {T<:Real,I<:Int,T1<:Function,T2<:Function,T3<:Function}
    @unpack ys,y_size,a_size,aGrid = economy
    @unpack ga,gc  = solution 
    
    
    # similar to what was done in the the function transitionMat
    # we define functions for savings (q_foo), durable and non-durable consumption (c_d_foo and c_nd_foo)    
    y_inds = 1:y_size
    a_inds = 1:a_size
    
    itr_a  = linear_interpolation((aGrid,y_inds),ga,extrapolation_bc=Interpolations.Flat())
    itr_c  = linear_interpolation((aGrid,y_inds),gc,extrapolation_bc=Interpolations.Flat())
    
    a_foo  = (iy′,a′) -> itr_a(a′,iy′)
    c_foo  = (iy′,a′) -> itr_c(a′,iy′)
        
    # initialization of output paths
    a_path = zeros(T,length(y_path)+1)
    c_path = zeros(T,length(y_path)+1)
    
    # initialization of svaings and durable consumption paths
    a_path[1],c_path[1] = a0,c0
    for (i,iy) in enumerate(y_path)
        a′          = a_path[i]
        a_path[i+1] = a_foo(iy,a′)
        c_path[i+1] = c_foo(iy,a′)
    end
    
    # returning the output
    return a_path,c_path
end;

## Computing a restricted transition matrix

The function 
> `restricted_trans(iy::I,solution::AiyagariSolution) where I<:Int`

returns the transition matrix and the stationary distribution corresponding to `solution` but restricted to idiosyncratic state `iy`.

In other words, multiplying $n$ times the obtained transition matrix  to the obtained stationary distribution allows one to obtain the stationary distribution of agents that have been at least for $n$ consecutive periods in state $n$.

In [ ]:
function restricted_trans(iy::I,solution::AiyagariSolution) where I<:Int
    transM = copy(Matrix(solution.transitMat)')
    statD  = copy(solution.stationaryDist)
    (na,ny)= size(statD)
    iy′    = iy
    for iy in 1:ny
        (iy==iy′) && continue
        for ia in 1:na
            statD[ia,iy] = zero(eltype(statD))
        end
    end
    for iy in 1:ny, jy in 1:ny
        (iy==iy′) && (jy==iy′) && continue
        for ia in 1:na, ja in 1:na
            transM[(iy-1)*na+ia,(jy-1)*na+ja] = zero(eltype(statD))
        end
    end
    return transM, statD
end

function restricted_trans(iy::I,jy::I,solution::AiyagariSolution) where I<:Int
    transM = copy(Matrix(solution.transitMat))
    statD  = copy(solution.stationaryDist)
    (na,ny)= size(statD)
    @show na,ny
    iy′    = iy
    jy′    = jy
    for iy in 1:ny
        (iy==iy′) && continue
        for ia in 1:na
            statD[ia,iy] = zero(eltype(statD))
        end
    end
    for iy in 1:ny, jy in 1:ny
        if (iy==iy′) && (jy==jy′)
            @show iy,jy,((iy-1)*na+1):iy*na,((jy-1)*na+1):jy*na
        end
        ((iy==iy′) && (jy==jy′)) && continue
        for ia in 1:na, ja in 1:na
            transM[(iy-1)*na+ia,(jy-1)*na+ja] = zero(eltype(statD))
        end
    end
    return transM, statD
end

In [ ]:
function transM_i_j(iy::I,jy::I,solution::AiyagariSolution) where I<:Int
    toR = similar(Matrix(solution.transitMat))
    (na,ny)= size(solution.stationaryDist)
    T = eltype(toR)
    toR .= zero(T)
    toR[((iy-1)*na+1):iy*na,((jy-1)*na+1):jy*na] = solution.transitMat[((iy-1)*na+1):iy*na,((jy-1)*na+1):jy*na]
    return toR
end
function statD_i(iy::I,solution::AiyagariSolution) where I<:Int
    toR = similar(Matrix(solution.stationaryDist))
    (na,ny)= size(toR)
    T = eltype(toR)
    toR .= zero(T)
    toR[:,iy] = solution.stationaryDist[:,iy]
    return toR
end

    

The function 
> `long_term_value(iy::I,solution::AiyagariSolution; policy::Symbol=:ga, tol=1e-6, maxiter=100) where I<:Int`

returns the value of the variable `policy` (by default asset, `:ga`) for agents remaining in state `iy` for a long period.

For instance, `long_term_value(iy,solution, policy=:gc)` returns the consumption level of agents remaining for a long time in state `iy`. 

In [ ]:
function long_term_value(iy::I, solution::AiyagariSolution; 
        policy::Symbol=:ga, tol=1e-6, maxiter=100) where I<:Int
    gx = getproperty(solution,policy)[:]
    transM_iy,statD_iy = restricted_trans(iy,solution);
    
    transM_iy_n = transM_iy^2500
    statD_iy_n  = transM_iy_n * statD_iy[:]
    if sum(statD_iy_n) ≈ zero(statD_iy_n[1])
        toR  = zero(gx[1])
    else
        toR  = sum(gx.*statD_iy_n)/sum(statD_iy_n)
    end
    @show toR
    toR_ = toR
    diff = one(eltype(gx))
    iter  = 0
    while (diff > tol)&&(iter<maxiter)        
        statD_iy_n = transM_iy_n * statD_iy_n
        toR_ = toR
        if sum(statD_iy_n) ≈ zero(statD_iy_n[1])
            toR  = zero(gx[1])
        else
            toR  = sum(gx.*statD_iy_n)/sum(statD_iy_n)
        end
        diff = abs(toR-toR_)
        iter+= 1
    end
    return toR
end

In [ ]:
function long_term_path(iy::I, n::I, x0::T, solution::AiyagariSolution, economy::Economy;
         policy::Symbol=:ga, tol=1e-6, maxiter=100) where {I<:Int,T<:Real}
    @unpack aGrid, a_size, a_min = economy
    MS = solution.stationaryDist
    gx = getproperty(solution,policy)
    gx_foo(x) = interpLinear(x,aGrid,gx[:,iy],a_size,a_min)[1]
    
    toR = zeros(T,n)

    toR[1] = x0
    for i in eachindex(toR[2:end])
        toR[i+1] = gx_foo(toR[i])
    end
    return toR
end

# Deprecated functions  <a id="deprecated"></a>[<font size=1>(back to menu)</font>](#Utils)

These functions are not used anymore but kept for legacy reasons.

## Converting from base `E` to base 10 (and vice-versa)


### From base `E` to base 10

The function `convertBasisE10(vE, E, N)` converts a number expressed in base-`E` (and represented as the vector `vE`) into its decimal representation `p` (as an integer).

***Remarks*** 
* elements in `vE` are  between 1 and $E$ (and not 0 and $E-1$);
* `vE[1]` represents the largest power and `vE[N]` represents units.

In [3]:
function convertBasisE10(vE::Vector{I}, E::I)::I where {I<:Integer}
# converts the a vector vE into an integer p 

    @assert (minimum(vE)≥1)&&(maximum(vE)≤E)
    
    N = length(vE)    
    p = one(I) #otherwise p lieas between 0 and (E+1)^N - 1
    for k ∈ eachindex(vE)
        p += (vE[k]-1)*E^(N-k)
    end
    return p
end;

#### Example

We convert `[2, 2, 1]` (i.e., 110 with a more usual binary notation) into base 10.

In [7]:
convertBasisE10([2, 2, 1], 2)

7

### From base 10 to base `E`

The function `convertBasis10E(p, E, N)` converts a number expressed in base-10 (and represented as the integer p) into its base-`E` representation `vE` (as a vector of length `N`). This is the inverse of `convertBasisE10`.

Same remarks as for `convertBasisE10`.

In [10]:
function  convertBasis10E(p::I, E::I, N::I)::Vector{I} where{I<:Integer}

    vE = zeros(I, N)    
    @assert (p≥1)&&(p≤E^N)
    ptemp = p-1;
    for k ∈ eachindex(vE)
        ptemp, r = divrem(ptemp,E)
        vE[N-k+1] = 1+r        
    end
    return vE
end

convertBasis10E (generic function with 1 method)

#### Example

We check that 7 converts back into  `[2, 2, 1]`.

In [13]:
convertBasis10E(7, 2, 3)

3-element Vector{Int64}:
 2
 2
 1